# Creacion Consolidado Inventario

Comenzaremos el analisis de la data consolidando la info en un unico DataSet

In [1]:
import pandas as pd
import numpy as np
import os
import warnings

from tqdm import tqdm
from unicodedata import normalize
from pjud.data import transformdata
from pjud.data import cleandata

In [2]:
warnings.filterwarnings(action="ignore")
pd.set_option('display.max_columns', 100)

In [5]:
tqdm.pandas()

In [6]:
path_processed = "../data/processed/pjud"

In [8]:
df_inventario = pd.read_feather(f"{path_processed}/processes_Inventario.feather")
print(f"Existen : {len(df_inventario)} causas en inventario")

Existen : 1476086 causas en inventario


In [9]:
df_inventario.columns

Index(['index', 'COD. CORTE', 'CORTE', 'COD. TRIBUNAL', 'TRIBUNAL', 'RIT',
       'COMPETENCIA', 'TIPO CAUSA', 'COD. MATERIA', 'MATERIA',
       'TIPO ULT. DILIGENCIA', 'FECHA ULT. DILIGENCIA', 'FECHA INGRESO',
       'TOTAL INVENTARIO'],
      dtype='object')

Unificar RIT y Tribunal en una sola columna para evitar mala interpretacion de causas

In [10]:
df_inventario['TRIBUNAL-RIT'] = df_inventario['COD. TRIBUNAL'].map(str) + "-" + df_inventario['RIT'].map(str)

In [11]:
df_inventario.sample(8)

,index,COD. CORTE,CORTE,COD. TRIBUNAL,TRIBUNAL,RIT,COMPETENCIA,TIPO CAUSA,COD. MATERIA,MATERIA,TIPO ULT. DILIGENCIA,FECHA ULT. DILIGENCIA,FECHA INGRESO,TOTAL INVENTARIO,TRIBUNAL-RIT
1368533,235979,90,C.A. DE SANTIAGO,1223,CUARTO JUZGADO DE GARANTIA SANTIAGO,10859-2016,GARANTIA,Ordinaria,802,ROBO CON INTIMIDACION.,RESOLUCION,2016-09-26,2016-09-23,1,1223-10859-2016
1447056,314830,91,C.A. DE SAN MIGUEL,1234,DECIMOQUINTO JUZGADO DE GARANTIA SANTIAGO,3561-2017,GARANTIA,Ordinaria,13036,LESIONES LEVES.,RESOLUCION,2017-06-27,2017-06-27,1,1234-3561-2017
901990,87931,30,C.A. DE VALPARAISO,1057,JUZGADO DE GARANTIA QUILLOTA,1653-2016,GARANTIA,Ordinaria,825,INFRACCION ARTICULO 454 C.PENAL.,AUDIENCIA,2016-11-28,2016-08-09,1,1057-1653-2016
1459831,327650,91,C.A. DE SAN MIGUEL,1237,JUZGADO DE GARANTIA SAN BERNARDO,12090-2017,GARANTIA,Ordinaria,10099,OTROS DELITOS DE LA LEY DE CONTROL DE ARMAS (L...,RESOLUCION,2017-12-20,2017-12-18,1,1237-12090-2017
1114917,301390,91,C.A. DE SAN MIGUEL,1236,JUZGADO DE GARANTIA PUENTE ALTO,7714-2015,GARANTIA,Ordinaria,221,DELITOS CONTRA LA VIDA Y LA PRIVACIDAD DE LAS ...,RESOLUCION,2015-07-01,2015-07-01,1,1236-7714-2015
1023603,209744,90,C.A. DE SANTIAGO,1223,CUARTO JUZGADO DE GARANTIA SANTIAGO,11279-2012,GARANTIA,Ordinaria,803,ROBO CON VIOLENCIA.,RESOLUCION,2014-07-15,2012-08-16,1,1223-11279-2012
925994,111971,40,C.A. DE TALCA,964,JUZGADO DE GARANTIA TALCA,320-2015,GARANTIA,Ordinaria,7038,CONSUMO/PORTE EN LUG.PUB.O PRIV.C/PREVIO CONCI...,RESOLUCION,2015-01-13,2015-01-13,1,964-320-2015
1387781,255284,90,C.A. DE SANTIAGO,1226,SEPTIMO JUZGADO DE GARANTIA SANTIAGO,10838-2015,GARANTIA,Ordinaria,834,USURPACION NO VIOLENTA (ART. 458 CODIGO PENAL).,RESOLUCION,2015-06-08,2015-06-05,1,1226-10838-2015


Imputo algunos datos que si dispongo del dataset.

In [12]:
df_inventario['AÑO INGRESO'] = df_inventario['RIT'].progress_apply(cleandata.obtiene_año)




100%|██████████| 1476086/1476086 [00:02<00:00, 655331.31it/s]


In [13]:
df_inventario.columns

Index(['index', 'COD. CORTE', 'CORTE', 'COD. TRIBUNAL', 'TRIBUNAL', 'RIT',
       'COMPETENCIA', 'TIPO CAUSA', 'COD. MATERIA', 'MATERIA',
       'TIPO ULT. DILIGENCIA', 'FECHA ULT. DILIGENCIA', 'FECHA INGRESO',
       'TOTAL INVENTARIO', 'TRIBUNAL-RIT', 'AÑO INGRESO'],
      dtype='object')

In [14]:
columnas_duplicadas = ['index']

df_inventario.drop(columnas_duplicadas, axis='columns', inplace=True)



## Carga Data relacionada a Tipologia de delitos con INVENTARIO

In [15]:
path = "../data/processed/delitos"
df_tipologia = pd.read_feather(f"{path}/clean_Delitos.feather")
print(f"{len(df_tipologia)} delitos utilizados en PJUD")

542 delitos utilizados en PJUD


In [16]:
df_tipologia.columns

Index(['index', 'COD. MATERIA', 'MATERIA', 'TIPOLOGIA MATERIA',
       'VIGENCIA MATERIA'],
      dtype='object')

In [17]:
df_inventario_tipologia = pd.merge(df_inventario,df_tipologia, how='left', on=['COD. MATERIA'])

In [18]:
df_inventario_tipologia.sample(10)

,COD. CORTE,CORTE,COD. TRIBUNAL,TRIBUNAL,RIT,COMPETENCIA,TIPO CAUSA,COD. MATERIA,MATERIA_x,TIPO ULT. DILIGENCIA,FECHA ULT. DILIGENCIA,FECHA INGRESO,TOTAL INVENTARIO,TRIBUNAL-RIT,AÑO INGRESO,index,MATERIA_y,TIPOLOGIA MATERIA,VIGENCIA MATERIA
1468220,91,C.A. DE SAN MIGUEL,1238,JUZGADO DE GARANTIA MELIPILLA,2995-2015,GARANTIA,Ordinaria,803,ROBO CON VIOLENCIA.,RESOLUCION,2015-11-14,2015-11-04,1,1238-2995-2015,2015,142.0,"ROBO CON VIOLENCIA. ART.436 INC. 1º 433, 438,...",LIBRO II TITULO IX CRIMENES Y SIMPLES DELITOS ...,VIGENTE
388868,91,C.A. DE SAN MIGUEL,1230,UNDECIMO JUZGADO DE GARANTIA SANTIAGO,5298-2019,GARANTIA,Ordinaria,848,HURTO IMPLE POR UN VALOR DE MEDIA A MENO DE ...,RESOLUCION,2019-07-22,2019-06-17,1,1230-5298-2019,2019,174.0,HURTO SIMPLE POR UN VALOR DE MEDIA A MENOS DE...,LIBRO II TITULO IX CRIMENES Y SIMPLES DELITOS ...,VIGENTE
212084,46,C.A. DE CONCEPCION,1082,JUZGADO DE GARANTIA CONCEPCION,4053-2019,GARANTIA,Ordinaria,13036,LE IONE LEVE .,RESOLUCION,2019-09-26,2019-04-19,1,1082-4053-2019,2019,417.0,LESIONES LEVES 494 Nº 5 CODIGO PENAL,FALTAS,VIGENTE
1058047,90,C.A. DE SANTIAGO,1226,SEPTIMO JUZGADO DE GARANTIA SANTIAGO,3983-2014,GARANTIA,Ordinaria,848,HURTO SIMPLE POR UN VALOR DE MEDIA A MENOS DE ...,RESOLUCION,2015-07-14,2014-02-25,1,1226-3983-2014,2014,174.0,HURTO SIMPLE POR UN VALOR DE MEDIA A MENOS DE...,LIBRO II TITULO IX CRIMENES Y SIMPLES DELITOS ...,VIGENTE
972884,50,C.A. DE TEMUCO,937,JUZGADO DE GARANTIA TEMUCO,3862-2016,GARANTIA,Ordinaria,14052,CONDUC. ESTADO DE EBRIEDAD CON O SIN DAÑOS O L...,RESOLUCION,2016-05-06,2016-05-05,1,937-3862-2016,2016,287.0,CONDUCCION ESTADO DE EBRIEDAD CON O SIN DAÑOS...,LEY DE TRANSITO,VIGENTE
913966,35,C.A. DE RANCAGUA,1070,JUZGADO DE GARANTIA RANCAGUA,13358-2016,GARANTIA,Ordinaria,7099,OTROS DELITOS DE LA LEY 20.000.,RESOLUCION,2016-10-25,2016-10-25,1,1070-13358-2016,2016,239.0,OTROS DELITOS DE LA LEY 20.000,LEY 20.000 TRAFICO ILICITO DE ESTUPEFACIENTES ...,VIGENTE
36855,15,C.A. DE ANTOFAGASTA,952,JUZGADO DE GARANTIA ANTOFAGASTA,12722-2019,GARANTIA,Ordinaria,15002,CRIMENE Y IMPLE DELITO EGURIDAD INTERIOR ...,RESOLUCION,2019-11-18,2019-11-14,1,952-12722-2019,2019,366.0,CRIMENES Y SIMPLES DELITOS SEGURIDAD INTERIOR...,INFRACCION A OTROS TEXTOS LEGALES,VIGENTE
242442,50,C.A. DE TEMUCO,943,JUZGADO DE GARANTIA LAUTARO,1539-2019,GARANTIA,Ordinaria,12074,CONDUC.SIN LA LICENCIA DEBIDA ART 194 LEY DE T...,AUDIENCIA,2019-12-17,2019-09-25,1,943-1539-2019,2019,270.0,CONDUCCION SIN LA LICENCIA DEBIDA ART 194 LEY...,LEY DE TRANSITO,VIGENTE
56873,20,C.A. DE COPIAPO,955,JUZGADO DE GARANTIA COPIAPO,8118-2019,GARANTIA,Ordinaria,840,DAÑO IMPLE .,AUDIENCIA,2019-12-10,2019-12-10,1,955-8118-2019,2019,169.0,DAÑOS SIMPLES. ART. 487.,LIBRO II TITULO IX CRIMENES Y SIMPLES DELITOS ...,VIGENTE
796694,91,C.A. DE SAN MIGUEL,1237,JUZGADO DE GARANTIA SAN BERNARDO,11918-2018,GARANTIA,Ordinaria,848,HURTO SIMPLE POR UN VALOR DE MEDIA A MENOS DE ...,AUDIENCIA,2018-10-31,2018-10-31,1,1237-11918-2018,2018,174.0,HURTO SIMPLE POR UN VALOR DE MEDIA A MENOS DE...,LIBRO II TITULO IX CRIMENES Y SIMPLES DELITOS ...,VIGENTE


In [19]:
columnas_duplicadas = ['index', 'MATERIA_x']

df_inventario_tipologia.drop(columnas_duplicadas, axis='columns', inplace=True)



In [20]:
df_inventario_tipologia.rename(columns = {'MATERIA_y':'MATERIA'}, inplace=True)

In [21]:
df_inventario_tipologia.columns

Index(['COD. CORTE', 'CORTE', 'COD. TRIBUNAL', 'TRIBUNAL', 'RIT',
       'COMPETENCIA', 'TIPO CAUSA', 'COD. MATERIA', 'TIPO ULT. DILIGENCIA',
       'FECHA ULT. DILIGENCIA', 'FECHA INGRESO', 'TOTAL INVENTARIO',
       'TRIBUNAL-RIT', 'AÑO INGRESO', 'MATERIA', 'TIPOLOGIA MATERIA',
       'VIGENCIA MATERIA'],
      dtype='object')

## Carga Data relacionada a Poblacion

In [22]:
df_poblacion = pd.read_feather(f"{path_processed}/processes_DataConsolidada_Poblacion_Jurisdiccion.feather")
print(f"{len(df_poblacion)} registros")

197 registros


In [23]:
df_poblacion

,index,REGION,CORTE,TRIBUNAL,POBLACION,HOMBRES,MUJERES,URBANO,RURAL,COMUNAS,JUECES,ASIENTO,TIPO JUZGADO
0,0,REGION DE TARAPACA,C.A. DE IQUIQUE,TRIBUNAL DE JUICIO ORAL EN LO PENAL IQUIQUE,330558,167793,162765,310065,20493,"[HUARA, CAMIÑA, COLCHANE, IQUIQUE, POZO ALMONT...",13,IQUIQUE,ORAL
1,1,REGION DE ANTOFAGASTA,C.A. DE ANTOFAGASTA,TRIBUNAL DE JUICIO ORAL EN LO PENAL CALAMA,177048,92417,84631,164011,13037,"[CALAMA, OLLAGUE, SAN PEDRO DE ATACAMA]",7,CALAMA,ORAL
2,2,REGION DE ANTOFAGASTA,C.A. DE ANTOFAGASTA,TRIBUNAL DE JUICIO ORAL EN LO PENAL ANTOFAGASTA,430486,222597,207889,407737,22749,"[TOCOPILLA, MARIA ELENA, MEJILLONES, SIERRA GO...",13,ANTOFAGASTA,ORAL
3,3,REGION DE ATACAMA,C.A. DE COPIAPO,TRIBUNAL DE JUICIO ORAL EN LO PENAL COPIAPO,286168,144420,141748,260520,25648,"[CHAÑARAL, DIEGO DE ALMAGRO, CALDERA, COPIAPO,...",9,COPIAPO,ORAL
4,4,REGION DE COQUIMBO,C.A. DE LA SERENA,TRIBUNAL DE JUICIO ORAL EN LO PENAL LA SERENA,496337,239495,256842,443484,52853,"[LA HIGUERA, VICUÑA, LA SERENA, COQUIMBO, ANDA...",10,LA SERENA,ORAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,192,REGION DE AISEN DEL GENERAL CARLOS IBAÑEZ DEL ...,C.A. DE COYHAIQUE,JUZGADO DE LETRAS Y GARANTIA COCHRANE,4638,2561,2077,2841,1797,"[COCHRANE, O'HIGGINS, TORTEL]",1,COCHRANE,LETRAS Y GARANTIA
193,193,REGION DE MAGALLANES Y ANTARTICA CHILENA,C.A. DE PUNTA ARENAS,JUZGADO DE LETRAS Y GARANTIA NATALES,22686,11792,10894,19180,3506,"[NATALES, TORRES DEL PAINE]",2,NATALES,LETRAS Y GARANTIA
194,194,REGION DE MAGALLANES Y ANTARTICA CHILENA,C.A. DE PUNTA ARENAS,JUZGADO DE LETRAS Y GARANTIA PORVENIR,8364,5266,3098,6062,2302,"[PORVENIR, PRIMAVERA, TIMAUKEL]",1,PORVENIR,LETRAS Y GARANTIA
195,195,REGION DE MAGALLANES Y ANTARTICA CHILENA,C.A. DE PUNTA ARENAS,JUZGADO DE LETRAS Y GARANTIA CABO DE HORNOS,2201,1321,880,1874,327,"[CABO DE HORNOS, ANTARTICA]",1,CABO DE HORNOS,LETRAS Y GARANTIA


In [24]:
df_fulldatainventario = pd.merge(df_inventario_tipologia, df_poblacion, how='left', on=['CORTE','TRIBUNAL'])

In [25]:
len(df_fulldatainventario)

1476086

In [26]:
df_fulldatainventario

,COD. CORTE,CORTE,COD. TRIBUNAL,TRIBUNAL,RIT,COMPETENCIA,TIPO CAUSA,COD. MATERIA,TIPO ULT. DILIGENCIA,FECHA ULT. DILIGENCIA,FECHA INGRESO,TOTAL INVENTARIO,TRIBUNAL-RIT,AÑO INGRESO,MATERIA,TIPOLOGIA MATERIA,VIGENCIA MATERIA,index,REGION,POBLACION,HOMBRES,MUJERES,URBANO,RURAL,COMUNAS,JUECES,ASIENTO,TIPO JUZGADO
0,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,123-2016,ORAL,Ordinaria,856,AUDIENCIA,2019-10-31,2016-04-29,1,988-123-2016,2016,APROPIACION INDEBIDA ART. 471 Nº 1.,LIBRO II TITULO IX CRIMENES Y SIMPLES DELITOS ...,VIGENTE,29,REGION DE ARICA Y PARINACOTA,226068,112581,113487,207231,18837,"[GENERAL LAGOS, PUTRE, ARICA, CAMARONES]",10,ARICA,ORAL
1,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,159-2019,ORAL,Ordinaria,7037,RESOLUCION,2019-12-31,2019-05-28,1,988-159-2019,2019,MICROTRAFICO (TRAFICO DE PEQUEÑAS CANTIDADES ...,LEY 20.000 TRAFICO ILICITO DE ESTUPEFACIENTES ...,VIGENTE,29,REGION DE ARICA Y PARINACOTA,226068,112581,113487,207231,18837,"[GENERAL LAGOS, PUTRE, ARICA, CAMARONES]",10,ARICA,ORAL
2,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,160-2019,ORAL,Ordinaria,10011,RESOLUCION,2019-11-21,2019-05-28,1,988-160-2019,2019,"POSESION, TENENCIA O PORTE DE MUNICIONES Y SU...",LEY 17.798 CONTROL DE ARMAS,VIGENTE,29,REGION DE ARICA Y PARINACOTA,226068,112581,113487,207231,18837,"[GENERAL LAGOS, PUTRE, ARICA, CAMARONES]",10,ARICA,ORAL
3,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,176-2019,ORAL,Ordinaria,12051,RESOLUCION,2019-12-31,2019-06-06,1,988-176-2019,2019,CONTRABANDO. INFRACCION A LA ORDENANZA DE ADU...,INFRACCION A OTROS TEXTOS LEGALES,VIGENTE,29,REGION DE ARICA Y PARINACOTA,226068,112581,113487,207231,18837,"[GENERAL LAGOS, PUTRE, ARICA, CAMARONES]",10,ARICA,ORAL
4,10,C.A. DE ARICA,988,TRIBUNAL DE JUICIO ORAL EN LO PENAL ARICA,196-2013,ORAL,Ordinaria,7007,RESOLUCION,2019-09-10,2013-08-05,1,988-196-2013,2013,TRAFICO ILICITO DE DROGAS ART. 3 LEY Nº 20.000.,LEY 20.000 TRAFICO ILICITO DE ESTUPEFACIENTES ...,VIGENTE,29,REGION DE ARICA Y PARINACOTA,226068,112581,113487,207231,18837,"[GENERAL LAGOS, PUTRE, ARICA, CAMARONES]",10,ARICA,ORAL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1476081,91,C.A. DE SAN MIGUEL,1355,TRIBUNAL DE JUICIO ORAL EN LO PENAL MELIPILLA,87-2017,ORAL,Ordinaria,10008,RESOLUCION,2017-12-14,2017-07-24,1,1355-87-2017,2017,PORTE DE ARMA PROHIBIDA (ART. 14 INC. 1°).,LEY 17.798 CONTROL DE ARMAS,VIGENTE,33,REGION METROPOLITANA DE SANTIAGO,185966,93110,92856,114535,71431,"[MELIPILLA, SAN PEDRO, ALHUE, CURACAVI, MARIA ...",6,MELIPILLA,ORAL
1476082,91,C.A. DE SAN MIGUEL,1355,TRIBUNAL DE JUICIO ORAL EN LO PENAL MELIPILLA,89-2017,ORAL,Ordinaria,12149,RESOLUCION,2017-12-06,2017-07-26,1,1355-89-2017,2017,DESACATO ART. 240 CODIGO DE PROCEDIMIENTO CIVIL,INFRACCION A OTROS TEXTOS LEGALES,VIGENTE,33,REGION METROPOLITANA DE SANTIAGO,185966,93110,92856,114535,71431,"[MELIPILLA, SAN PEDRO, ALHUE, CURACAVI, MARIA ...",6,MELIPILLA,ORAL
1476083,91,C.A. DE SAN MIGUEL,1355,TRIBUNAL DE JUICIO ORAL EN LO PENAL MELIPILLA,91-2017,ORAL,Ordinaria,623,RESOLUCION,2017-12-11,2017-07-28,1,1355-91-2017,2017,ABUSO SEXUAL CON CONTACTO CORPORAL DE MENOR DE...,LIBRO II TITULO VII CRIMENES Y SIMPLES DELIT...,VIGENTE,33,REGION METROPOLITANA DE SANTIAGO,185966,93110,92856,114535,71431,"[MELIPILLA, SAN PEDRO, ALHUE, CURACAVI, MARIA ...",6,MELIPILLA,ORAL
1476084,91,C.A. DE SAN MIGUEL,1355,TRIBUNAL DE JUICIO ORAL EN LO PENAL MELIPILLA,95-2017,ORAL,Ordinaria,7037,RESOLUCION,2017-12-18,2017-08-02,1,1355-95-2017,2017,MICROTRAFICO (TRAFICO DE PEQUEÑAS CANTIDADES ...,LEY 20.000 TRAFICO ILICITO DE ESTUPEFACIENTES ...,VIGENTE,33,REGION METROPOLITANA DE SANTIAGO,185966,93110,92856,114535,71431,"[MELIPILLA, SAN PEDRO, ALHUE, CURACAVI, MARIA ...",6,MELIPILLA,ORAL


In [27]:
df_fulldatainventario.columns

Index(['COD. CORTE', 'CORTE', 'COD. TRIBUNAL', 'TRIBUNAL', 'RIT',
       'COMPETENCIA', 'TIPO CAUSA', 'COD. MATERIA', 'TIPO ULT. DILIGENCIA',
       'FECHA ULT. DILIGENCIA', 'FECHA INGRESO', 'TOTAL INVENTARIO',
       'TRIBUNAL-RIT', 'AÑO INGRESO', 'MATERIA', 'TIPOLOGIA MATERIA',
       'VIGENCIA MATERIA', 'index', 'REGION', 'POBLACION', 'HOMBRES',
       'MUJERES', 'URBANO', 'RURAL', 'COMUNAS', 'JUECES', 'ASIENTO',
       'TIPO JUZGADO'],
      dtype='object')

VERIFICAR FILAS A ELIMINAR ... CAMBIARON ....

In [28]:
columnas_duplicadas = ['index']

df_fulldatainventario.drop(columnas_duplicadas, axis='columns', inplace=True)


In [29]:
#df_consolidado.drop(['MATERIA_x','FECHA TERMINO_x','AÑO TERMINO_x','TIPO CAUSA_y'], axis='columns', inplace=True)

In [30]:
df_fulldatainventario.sample(5)

,COD. CORTE,CORTE,COD. TRIBUNAL,TRIBUNAL,RIT,COMPETENCIA,TIPO CAUSA,COD. MATERIA,TIPO ULT. DILIGENCIA,FECHA ULT. DILIGENCIA,FECHA INGRESO,TOTAL INVENTARIO,TRIBUNAL-RIT,AÑO INGRESO,MATERIA,TIPOLOGIA MATERIA,VIGENCIA MATERIA,REGION,POBLACION,HOMBRES,MUJERES,URBANO,RURAL,COMUNAS,JUECES,ASIENTO,TIPO JUZGADO
6966,10,C.A. DE ARICA,992,JUZGADO DE GARANTIA ARICA,6878-2019,GARANTIA,Ordinaria,14052,RESOLUCION,2019-12-10,2019-07-03,1,992-6878-2019,2019,CONDUCCION ESTADO DE EBRIEDAD CON O SIN DAÑOS...,LEY DE TRANSITO,VIGENTE,REGION DE ARICA Y PARINACOTA,226068,112581,113487,207231,18837,"[GENERAL LAGOS, PUTRE, ARICA, CAMARONES]",5,ARICA,GARANTIA
1456143,91,C.A. DE SAN MIGUEL,1236,JUZGADO DE GARANTIA PUENTE ALTO,6184-2017,GARANTIA,Ordinaria,710,RESOLUCION,2017-12-29,2017-05-14,1,1236-6184-2017,2017,LESIONES MENOS GRAVES. ART. 399.,LIBRO II TITULO VIII CRIMENES Y SIMPLES DELIT...,VIGENTE,REGION METROPOLITANA DE SANTIAGO,612816,298437,314379,590972,21844,"[PUENTE ALTO, SAN JOSE DE MAIPO, PIRQUE]",8,PUENTE ALTO,GARANTIA
964549,46,C.A. DE CONCEPCION,1083,JUZGADO DE GARANTIA LOS ANGELES,5525-2016,GARANTIA,Ordinaria,13028,RESOLUCION,2016-11-04,2016-11-04,1,1083-5525-2016,2016,HURTO FALTA 494 BIS CODIGO PENAL,FALTAS,VIGENTE,REGION DEL BIO BIO,215991,104675,111316,158750,57241,"[LOS ANGELES, QUILLECO, ANTUCO]",4,LOS ANGELES,GARANTIA
421323,91,C.A. DE SAN MIGUEL,1237,JUZGADO DE GARANTIA SAN BERNARDO,4787-2019,GARANTIA,Ordinaria,808,RESOLUCION,2019-04-18,2019-04-16,1,1237-4787-2019,2019,ROBO EN BIENES NACIONALES DE USO PUBLICO O SI...,LIBRO II TITULO IX CRIMENES Y SIMPLES DELITOS ...,VIGENTE,REGION METROPOLITANA DE SANTIAGO,496078,244614,251464,437457,58621,"[SAN BERNARDO, CALERA DE TANGO, BUIN, PAINE]",10,SAN BERNARDO,GARANTIA
345011,90,C.A. DE SANTIAGO,1226,SEPTIMO JUZGADO DE GARANTIA SANTIAGO,3585-2018,GARANTIA,Ordinaria,848,RESOLUCION,2018-02-26,2018-02-26,1,1226-3585-2018,2018,HURTO SIMPLE POR UN VALOR DE MEDIA A MENOS DE...,LIBRO II TITULO IX CRIMENES Y SIMPLES DELITOS ...,VIGENTE,REGION METROPOLITANA DE SANTIAGO,404495,206678,197817,404495,0,[SANTIAGO],12,SANTIAGO,GARANTIA


Reordenando Nombres de las columnas ...

In [36]:
df_fulldatainventario.rename(columns = {'COD. CORTE':'cod_corte',
                                        'COD. TRIBUNAL':'cod_tribunal',
                                         'RIT':'rit',
                                         'CORTE':'corte',
                                         'TRIBUNAL':'tribunal',
                                         'COMPETENCIA':'competencia',
                                         'TIPO CAUSA':'tipo_causa',
                                         'COD. MATERIA':'cod_materia',
                                         'TIPO ULT. DILIGENCIA':'tipo_ultima_diligencia',
                                         'FECHA ULT. DILIGENCIA':'fecha_ultima_diligencia',
                                         'FECHA INGRESO':'fecha_ingreso',
                                         'AÑO INGRESO':'año_ingreso',
                                         'TOTAL INVENTARIO':'total_inventario',
                                         'TRIBUNAL-RIT':'tribunal_rit',
                                         'MATERIA':'materia',
                                         'TIPOLOGIA MATERIA':'tipologia_materia',
                                         'VIGENCIA MATERIA':'vigencia_materia',
                                         'REGION':'region',
                                         'POBLACION':'poblacion',
                                         'HOMBRES':'hombres',
                                         'MUJERES':'mujeres',
                                         'URBANO':'urbano',
                                         'RURAL':'rural',
                                         'COMUNAS':'comunas',
                                         'JUECES':'dotacion_jueces',
                                         'ASIENTO':'asiento',
                                         'TIPO JUZGADO':'tipo_juzgado'
                                     },inplace = True)

In [38]:
df_fulldatainventario.columns

Index(['cod_corte', 'corte', 'cod_tribunal', 'tribunal', 'rit', 'competencia',
       'tipo_causa', 'cod_materia', 'tipo_ultima_diligencia',
       'fecha_ultima_diligencia', 'fecha_ingreso', 'total_inventario',
       'tribunal_rit', 'año_ingreso', 'materia', 'tipologia_materia',
       'vigencia_materia', 'region', 'poblacion', 'hombres', 'mujeres',
       'urbano', 'rural', 'comunas', 'dotacion_jueces', 'asiento',
       'tipo_juzgado'],
      dtype='object')

In [40]:
df_fulldatainventario = df_fulldatainventario[['region','cod_corte','corte','tribunal_rit','cod_tribunal','rit','tribunal','competencia','tipo_juzgado','dotacion_jueces','tipo_causa',
                                               'año_ingreso','fecha_ingreso','cod_materia','materia','tipologia_materia','vigencia_materia','tipo_ultima_diligencia','fecha_ultima_diligencia',
                                               'total_inventario','asiento','comunas','poblacion','hombres','mujeres','urbano','rural']]

In [41]:
df_fulldatainventario.sample(5)

,region,cod_corte,corte,tribunal_rit,cod_tribunal,rit,tribunal,competencia,tipo_juzgado,dotacion_jueces,tipo_causa,año_ingreso,fecha_ingreso,cod_materia,materia,tipologia_materia,vigencia_materia,tipo_ultima_diligencia,fecha_ultima_diligencia,total_inventario,asiento,comunas,poblacion,hombres,mujeres,urbano,rural
806058,REGION METROPOLITANA DE SANTIAGO,91,C.A. DE SAN MIGUEL,1238-1216-2017,1238,1216-2017,JUZGADO DE GARANTIA MELIPILLA,GARANTIA,GARANTIA,3,Ordinaria,2017,2017-03-28,840,DAÑOS SIMPLES. ART. 487.,LIBRO II TITULO IX CRIMENES Y SIMPLES DELITOS ...,VIGENTE,RESOLUCION,2017-06-14,1,MELIPILLA,"[MELIPILLA, SAN PEDRO, ALHUE]",139797,70058,69739,87508,52289
1006277,DUOREGION DE MAGALLANES Y LA ANTARTICA CHILENA,61,C.A. DE PUNTA ARENAS,995-3572-2016,995,3572-2016,JUZGADO DE GARANTIA PUNTA ARENAS,GARANTIA,GARANTIA,4,Ordinaria,2016,2016-11-30,710,LESIONES MENOS GRAVES. ART. 399.,LIBRO II TITULO VIII CRIMENES Y SIMPLES DELIT...,VIGENTE,RESOLUCION,2016-12-01,1,PUNTA ARENAS,"[LAGUNA BLANCA, SAN GREGORIO, RIO VERDE, PUNTA...",133282,66870,66412,125932,7350
679449,REGION METROPOLITANA DE SANTIAGO,90,C.A. DE SANTIAGO,1220-357-2013,1220,357-2013,PRIMER JUZGADO DE GARANTIA SANTIAGO,GARANTIA,GARANTIA,5,Ordinaria,2013,2013-01-25,710,LESIONES MENOS GRAVES. ART. 399.,LIBRO II TITULO VIII CRIMENES Y SIMPLES DELIT...,VIGENTE,AUDIENCIA,2013-05-10,1,SANTIAGO,[PUDAHUEL],230293,112412,117881,226138,4155
104992,REGION DE VALPARAISO,30,C.A. DE VALPARAISO,1049-2982-2017,1049,2982-2017,JUZGADO DE GARANTIA SAN ANTONIO,GARANTIA,GARANTIA,4,Ordinaria,2017,2017-06-20,8002,DELITOS MARCARIOS.ART. 28.LEY Nº 19.039.,LEY 19.039 DE PROPIEDAD Y PRIVILEGIOS INDUSTRI...,VIGENTE,RESOLUCION,2017-06-29,1,SAN ANTONIO,"[ALGARROBO, EL QUISCO, EL TABO, CARTAGENA, SAN...",168046,82648,85398,152012,16034
930639,REGION DEL MAULE,40,C.A. DE TALCA,965-1475-2015,965,1475-2015,JUZGADO DE GARANTIA SAN JAVIER,GARANTIA,GARANTIA,1,Ordinaria,2015,2015-10-01,12147,EJERCICIO IRREGULAR DE MARTILLERO PUBLICO ART...,INFRACCION A OTROS TEXTOS LEGALES,VIGENTE,RESOLUCION,2016-12-27,1,SAN JAVIER,"[SAN JAVIER, VILLA ALEGRE]",61768,29882,31886,42245,19523


In [42]:
# Directorio donde se guardaran archivos feather
df_fulldatainventario.reset_index(inplace = True)
os.makedirs(path_processed, exist_ok = True)  

# Guardamos dataset como archivo feather

df_fulldatainventario.to_feather(f'{path_processed}/consolidated_FULLDATA_INVENTARIOS.feather')